In [ ]:
import torchvision

from torch.utils import data
from torchvision import transforms


def load_data_fashion_mnist(batch_size, resize=None):  # @save
    """下载Fashion-MNIST数据集，然后将其加载到内存中"""
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0, transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(
        root="data",
        train=True,
        transform=trans,
        download=True,
    )
    mnist_test = torchvision.datasets.FashionMNIST(
        root="data",
        train=False,
        transform=trans,
        download=True,
    )
    return (
        data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=4),
        data.DataLoader(mnist_test, batch_size, shuffle=False, num_workers=4),
    )


batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size)